In [14]:
import sys
import os
import torch
import glob
import pandas as pd
import numpy as np
from sklearn import metrics,preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
import collections

import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data_utils

# Imports for PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
# For full dataset, run tran_feature_selection notebook before this one
# For small subset of data, run tran_sample_preprocessing notebook beofore this one (ideal for testing model on CPU)

%store -r benign_flows  
%store -r mixed_flows
%store -r features

dim = len(features)

In [3]:
x = benign_flows.loc[:, features].values
x = StandardScaler().fit_transform(x) # normalizing the features
np.mean(x),np.std(x)

(3.85338091401729e-16, 0.9999999999999999)

In [4]:
# finding the top k components whose cumulative variance exceeds 95%
  
pca = PCA(0.95)
pca.fit(x)
top_k_component = pca.n_components_
top_k_component


34

In [5]:
# re-run PCA with only top k features again
top_k_features = features[:top_k_component]

top_k_x = benign_flows.loc[:, top_k_features].values
top_k_x = StandardScaler().fit_transform(top_k_x) # normalizing the features
np.mean(top_k_x),np.std(top_k_x)

top_k_pca = PCA(n_components=top_k_component-1)   
top_k_principalComponents = top_k_pca.fit_transform(top_k_x)

top_k_var=np.cumsum(np.round(top_k_pca.explained_variance_ratio_, decimals=3)*100)
# print(top_k_var)


In [6]:
# inverse mapping of low dimensional to original coordinate space
data_reduced = np.dot(x-pca.mean_, pca.components_.T) # transform
data_original = np.dot(data_reduced, pca.components_)+ pca.mean_ # inverse_transform
# print(data_original-x)  #reconstruction error

In [142]:
labels = ['is_attack']

x_train = x
benign_flows['is_attack']= 0
y_train = benign_flows.loc[:, labels].values

x_test = mixed_flows.loc[:, features].values
y_test = mixed_flows.loc[:, labels].values    

# normalizing
#x_train = preprocessing.normalize(x_train, axis=0)
x_test = preprocessing.normalize(x_train, axis=0)

In [144]:
train_pca = pca.transform(x_train_flat)
test_pca = pca.transform(x_test_flat)

In [12]:
# necessary libraries to run the deep learning model
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.optimizers import RMSprop

In [145]:
#constant declaration
batch_size = 128
num_classes  = 2
epochs = 10

In [146]:
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(top_k_component,)))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# model.summary()

In [147]:
# compiling and training the model with the reduced dimension
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(train_pca, y_train,batch_size=batch_size,epochs=epochs,verbose=1,
                    validation_data=(test_pca, y_test))

Epoch 1/10
391/391 [==============================] - 14s 34ms/step - loss: 0.0491 - accuracy: 0.9850 - val_loss: 363790848.0000 - val_accuracy: 0.3886
Epoch 2/10
391/391 [==============================] - 13s 33ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 363790848.0000 - val_accuracy: 0.3886
Epoch 3/10
391/391 [==============================] - 13s 33ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 363790848.0000 - val_accuracy: 0.3886
Epoch 4/10
391/391 [==============================] - 14s 35ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 363790848.0000 - val_accuracy: 0.3886
Epoch 5/10
391/391 [==============================] - 14s 35ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 363790848.0000 - val_accuracy: 0.3886
Epoch 6/10
391/391 [==============================] - 13s 34ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 363790848.0000 - val_accuracy: 0.3886
Epoch 7/10
391/391 [==============================] - 15s 39ms/step 

In [148]:
# checking how the model works with all the features
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(dim,)))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,
                    validation_data=(x_test, y_test))

Epoch 1/10
391/391 [==============================] - 14s 35ms/step - loss: 0.0154 - accuracy: 0.9883 - val_loss: 5.2240 - val_accuracy: 0.3886
Epoch 2/10
391/391 [==============================] - 14s 35ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 5.2240 - val_accuracy: 0.3886
Epoch 3/10
391/391 [==============================] - 14s 36ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 5.2240 - val_accuracy: 0.3886
Epoch 4/10
391/391 [==============================] - 13s 34ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 5.2240 - val_accuracy: 0.3886
Epoch 5/10
391/391 [==============================] - 14s 35ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 5.2240 - val_accuracy: 0.3886
Epoch 6/10
391/391 [==============================] - 14s 35ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 5.2240 - val_accuracy: 0.3886
Epoch 7/10
391/391 [==============================] - 14s 35ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss